In [1]:
%pwd

'/content'

**ChainerとCupyインストール**

In [2]:
!curl https://colab.chainer.org/install | sh -

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1580  100  1580    0     0   2867      0 --:--:-- --:--:-- --:--:--  2872
+ apt -y -q install cuda-libraries-dev-10-0
Reading package lists...
Building dependency tree...
Reading state information...
cuda-libraries-dev-10-0 is already the newest version (10.0.130-1).
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.
+ pip install -q cupy-cuda100  chainer 
     |████████████████████████████████| 369.6MB 45kB/s 
+ set +ex
Installation succeeded!


In [3]:
import numpy as np
import six

import chainer
from chainer import backend
from chainer.backends import cuda
from chainer import function
from chainer import utils
from chainer.utils import collections_abc
from chainer.utils import type_check

/usr/local/lib/python3.6/dist-packages/chainer/backends/cuda.py:147: UserWarning: cuDNN is not enabled.
Please reinstall CuPy after you install cudnn
(see https://docs-cupy.chainer.org/en/stable/install.html#install-cudnn).
  'cuDNN is not enabled.\n'


In [4]:
def flip_path(path, path_length, xp):
    """Flips label sequence.
    This function rotates a label sequence and flips it.
    ``path[b, t]`` stores a label at time ``t`` in ``b``-th batch.
    The rotated matrix ``r`` is defined as
    ``r[b, t] = path[b, t + path_length[b]]``
    .. ::
       a b c d .     . a b c d    d c b a .
       e f . . .  -> . . . e f -> f e . . .
       g h i j k     g h i j k    k j i h g
    """
    n_batch, n_label = path.shape
    rotate = (xp.arange(n_label) + path_length[:, None]) % n_label
    return path[xp.arange(n_batch, dtype=xp.int32)[:, None], rotate][:, ::-1]

In [5]:
def flip_label_probability(y, input_length, xp):
    """Flips a label probability matrix.
    This function rotates a label probability matrix and flips it.
    ``y[i, b, l]`` stores log probability of label ``l`` at ``i``-th
    input in ``b``-th batch.
    The rotated matrix ``r`` is defined as
    ``r[i, b, l] = y[i + input_length[b], b, l]``
    """
    seq, n_batch, n_vocab = y.shape
    rotate = (xp.arange(seq, dtype=xp.int32)[:, None] + input_length) % seq
    return y[
        rotate[:, :, None],
        xp.arange(n_batch, dtype=xp.int32)[None, :, None],
        xp.arange(n_vocab, dtype=xp.int32)[None, None, :]][::-1]

In [6]:
def flip_path_probability(prob, input_length, path_length, xp):
    """Flips a path probability matrix.
    This function returns a path probability matrix and flips it.
    ``prob[i, b, t]`` stores log probability at ``i``-th input and
    at time ``t`` in a output sequence in ``b``-th batch.
    The rotated matrix ``r`` is defined as
    ``r[i, j, k] = prob[i + input_length[j], j, k + path_length[j]]``
    """
    seq, n_batch, n_label = prob.shape
    rotate_input = ((xp.arange(seq, dtype=xp.int32)[:, None] + input_length)
                    % seq)
    rotate_label = ((xp.arange(n_label, dtype=xp.int32) + path_length[:, None])
                    % n_label)
    return prob[
        rotate_input[:, :, None],
        xp.arange(n_batch, dtype=xp.int32)[None, :, None],
        rotate_label][::-1, :, ::-1]

In [7]:
def _logsumexp(a, xp, axis=None):
    #print('a = {}'.format(a))
    vmax = xp.amax(a, axis=axis, keepdims=True) 
    print('vmax = {}'.format(vmax))
    vmax += xp.log(xp.sum(xp.exp(a - vmax), axis=axis, keepdims=True, dtype=a.dtype))  
    #print('vmax = {}'.format(vmax))
    return xp.squeeze(vmax, axis=axis)


動的計画法

In [8]:
def computes_transition(i, prev_prob, path, path_length, cum_prob, y):
    n_batch, max_path_length = path.shape
    zero_padding = -10000.0

    mat = xp.full((2, n_batch, max_path_length), zero_padding, y.dtype)
    mat[0, :, :] = prev_prob              # a(t-1)(s)
    if i > 0:
      mat[1, :, 1:] = prev_prob[:, :-1]     # a(t-1)(s-1)
    print(('mat=\n{}'.format(mat)))

    #same_transition = (path[:, :-2] == path[:, 2:])
    #print('same_transition=\n{}'.format(same_transition))
    #mat[2, :, 2:][same_transition] = zero_padding
    #print('mat=\n{}'.format(mat))
    prob = _logsumexp(mat, xp, axis=0)    # logSumExp{a(t-1)(s)+a(t-1)(s-1)}
    print('prob=\n{}'.format(prob))

    outside = xp.arange(max_path_length) >= path_length[:, None]
    print('outside=\n{}'.format(outside))
    prob[outside] = zero_padding
    print('prob=\n{}'.format(prob))
    cum_prob += prob
    batch_index = xp.arange(n_batch, dtype=xp.int32)
    prob += y[batch_index[:, None], path]  # logy(t) + logSumExp{a(t-1)(s)+a(t-1)(s-1)}
    print('prob={}'.format(prob))
    
    return prob

In [9]:
#def calc_trans(yseq, input_length, label, label_length, path, path_length, xp):
def calc_trans(yseq, input_length, path, path_length, xp):
    #max_input_length, n_batch, n_unit = yseq.shape
    #max_path_length = path.shape[1]

    max_input_length, n_batch, n_unit = yseq.shape
    max_path_length = path.shape[1]
    zero_padding = -10000.0

    forward_prob = xp.full((n_batch, max_path_length), zero_padding, dtype=yseq.dtype)
    forward_prob[:, 0] = 0  # t=0のとき用
    print('forward_prob={}'.format(forward_prob))
    backward_prob = forward_prob

    batch_index = xp.arange(n_batch, dtype=xp.int32)
    print('batch_index={}'.format(batch_index))
    seq_index = xp.arange(len(yseq), dtype=xp.int32)
    print('seq_index={}'.format(seq_index))
    prob = yseq[seq_index[:, None, None], batch_index[:, None], path]
    print('prob.shape={}'.format(prob.shape))
    print('prob=\n{}'.format(prob))


    print('--------------- forward_computation -------------------')
    print('path=\n{}'.format(path))
    print('path_length=\n{}'.format(path_length))
    # forward computation.
    for i, y in enumerate(yseq):
        print('i={}'.format(i))
        print('forward_prob=\n{}'.format(forward_prob))
        print('prob[{}]=\n{}'.format(i, prob[i]))
        print('y=\n{}'.format(y))
        forward_prob = computes_transition(i, forward_prob, path, path_length, prob[i], y) # 1つ目の関数

    r_path = flip_path(path, path_length, xp)  # 2つ目の関数
    print('r_path=\n{}'.format(r_path))
    yseq_inv = flip_label_probability(yseq, input_length, xp)  # 3つ目の関数
    print('yseq_inv=\n{}'.format(yseq_inv))
    prob = flip_path_probability(prob, input_length, path_length, xp)  #  4つ目の関数
    print('prob=\n{}'.format(prob))
    
    # backward computation.
    print('--------------- backward_computation -------------------')
    for i, y_inv in enumerate(yseq_inv):
        print('i={}'.format(i))
        print('backward_prob=\n{}'.format(forward_prob))
        print('prob[{}]=\n{}'.format(i, prob[i]))
        print('y_inv=\n{}'.format(y_inv))
        backward_prob = computes_transition(i, backward_prob, r_path, path_length, prob[i], y_inv)

    return flip_path_probability(prob, input_length, path_length, xp)

In [10]:
def _softmax(x, xp):
    val = xp.exp(x - xp.amax(x, axis=2, keepdims=True))
    val /= xp.sum(val, axis=2, keepdims=True)
    return val

In [11]:
def log_matrix(x, xp):
    res = np.ma.log(x).filled(fill_value=-10000.0)
    print('res=\n{}'.format(res))
    return res.astype(x.dtype, copy=False)

In [12]:
def _label_to_path(labels, blank_symbol, xp):
    path = xp.full((len(labels), labels.shape[1] * 2 + 1),
                   blank_symbol, dtype=np.int32)
    path[:, 1::2] = labels
    return path

In [13]:
# input_length=4, batch_size=2, num_class=5
#yseq = [[[1.2, 2.3, 1.4, -0.5, 2.2], [-0.1, 1.2, 0.4, 2.5, 3.2]],
#         [[0.5, 1.3, 2.2, 0.1, 2.4], [1.1, 2.2, 0.7, 1.4, 2.2]],
#         [[0.8, -1.5, 2.3, 1.2, 2.1], [0.9, 1.4, 0.6, 2.3, 1.0]],
#         [[0.2, -1.0, 1.3, 2.2, 0.1], [0.2, 1.0, 1.6, 1.3, 1.2]]]
yseq = [[[1.2, 2.3, 1.4, -0.5, 2.2]],
         [[0.5, 1.3, 2.2, 0.1, 2.4]],
         [[0.8, -1.5, 2.3, 1.2, 2.1]],
         [[0.2, -1.0, 1.3, 2.2, 0.1]]]

yseq_np = np.array(yseq)
xp = backend.get_array_module(yseq_np)
# batch_size=2, max_target_length=3
#label = [[2, 3, 4],
#         [1, 2, -1]]
label = [[2, 3, 4]]
label_np = np.array(label)
# batch_size=2
#input_length = [4, 4]
input_length = [4]
input_length_np = np.array(input_length)
# batch_size=2
#label_length = [3, 2]
label_length = [3]
label_length_np = np.array(label_length)

#path_length = 2 * label_length + 1
#path_length = [7, 5]
#path_length = [7]
#path_length_np = np.array(path_length)
yseq_softmax = _softmax(yseq_np, xp)
print('yseq_softmax=\n{}'.format(yseq_softmax))
log_yseq = log_matrix(yseq_softmax, xp)
print('log_yseq=\n{}'.format(log_yseq))
#path = _label_to_path(label_np, 0, xp)

print('yseq_softmax={}'.format(yseq_softmax))
print('input_length_np={}'.format(input_length_np))
print('label_np={}'.format(label_np))
print('label_length_np={}'.format(label_length_np))
#print('path={}'.format(path))
#print('path_length_np={}'.format(path_length_np))

#prob_trans = calc_trans(log_yseq, input_length_np, label_np, label_length_np, path, path_length_np, xp)
prob_trans = calc_trans(log_yseq, input_length_np, label_np, label_length_np, xp)
print('prob_trans=\n{}'.format(prob_trans.shape))
print('prob_trans=\n{}'.format(prob_trans))
loss = -_logsumexp(prob_trans[0], xp, axis=1)
print('loss=\n{}'.format(loss))

yseq_softmax=
[[[0.12305369 0.36967371 0.15029811 0.02247988 0.33449461]]

 [[0.06228317 0.13861373 0.34093477 0.04174965 0.41641867]]

 [[0.09308327 0.00933242 0.41717027 0.13886392 0.34155013]]

 [[0.07936978 0.02390572 0.23844    0.58646776 0.07181675]]]
res=
[[[-2.09513453 -0.99513453 -1.89513453 -3.79513453 -1.09513453]]

 [[-2.7760641  -1.9760641  -1.0760641  -3.1760641  -0.8760641 ]]

 [[-2.37426083 -4.67426083 -0.87426083 -1.97426083 -1.07426083]]

 [[-2.53363759 -3.73363759 -1.43363759 -0.53363759 -2.63363759]]]
log_yseq=
[[[-2.09513453 -0.99513453 -1.89513453 -3.79513453 -1.09513453]]

 [[-2.7760641  -1.9760641  -1.0760641  -3.1760641  -0.8760641 ]]

 [[-2.37426083 -4.67426083 -0.87426083 -1.97426083 -1.07426083]]

 [[-2.53363759 -3.73363759 -1.43363759 -0.53363759 -2.63363759]]]
yseq_softmax=[[[0.12305369 0.36967371 0.15029811 0.02247988 0.33449461]]

 [[0.06228317 0.13861373 0.34093477 0.04174965 0.41641867]]

 [[0.09308327 0.00933242 0.41717027 0.13886392 0.34155013]]

 [[